<a href="https://colab.research.google.com/github/furkannulu/GMKA_Furkan_ULU/blob/main/Odevi4_Python_Veri_Analizi_Egitimi__Furkan_ULU_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Güney Marmara ve TRAI
# <center> Python Veri Analizi Eğitimi
# <center> Ödev 4

### Veriyi özetleme

1.  3. Ders sonunda hazırladığınız 2021 ve 2022 yıllarına ait BIST30 ve EUR/TRY kur değerlerini içeren bileşke tablodan sadece ayın son işgününe ait değerleri içeren bir pivot tablo üretiniz.

In [1]:
from pandas.core.arrays.datetimes import warnings
#library
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
# BIST30 ve EUR/TRY verilerinin temini
from pandas.core.internals import concat
indirme_listesi = ['XU030.IS', 'EURTRY=X']
Veri = pd.DataFrame()
for i in indirme_listesi:
  data = yf.download(i, start = '2021-01-01', end = '2022-12-31')
  data = data['Adj Close'].rename(i)
  Veri = pd.concat([Veri,data], axis=1)
Veri

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,XU030.IS,EURTRY=X
2021-01-04,1652.900024,9.105680
2021-01-05,1640.400024,9.088820
2021-01-06,1657.699951,9.081770
2021-01-07,1678.199951,9.006140
2021-01-08,1694.400024,8.998500
...,...,...
2022-07-11,NaN,17.503019
2022-07-12,NaN,17.325560
2022-07-15,NaN,17.506651
2022-08-30,NaN,18.190140


In [5]:
# Veri ön işleme
Veri.columns = ['Kapanış (TRY)', 'EUR/TRY']
Veri = Veri[Veri['Kapanış (TRY)']>0]
Veri

,Kapanış (TRY),EUR/TRY
2021-01-04,1652.900024,9.105680
2021-01-05,1640.400024,9.088820
2021-01-06,1657.699951,9.081770
2021-01-07,1678.199951,9.006140
2021-01-08,1694.400024,8.998500
...,...,...
2022-12-23,5885.200195,19.791670
2022-12-27,5876.700195,19.815269
2022-12-28,5789.399902,19.805389
2022-12-29,5915.200195,19.867519


In [6]:
# Normalizasyon
Veri['Normalize'] = Veri['Kapanış (TRY)']/Veri.loc[Veri.index[0],'Kapanış (TRY)']
Veri['Normalize EUR/TRY'] = Veri['EUR/TRY']/Veri.loc[Veri.index[0],'EUR/TRY']
Veri['degisim'] = Veri['Normalize'].diff(1)
Veri['d_yon'] = Veri['degisim'].map(lambda x: 'düşen gün' if x<0 else 'yükselen gün')
Veri['degisim%'] = Veri['degisim']/Veri['Normalize'].shift(1)*100
Veri

,Kapanış (TRY),EUR/TRY,Normalize,Normalize EUR/TRY,degisim,d_yon,degisim%
2021-01-04,1652.900024,9.105680,1.000000,1.000000,NaN,yükselen gün,NaN
2021-01-05,1640.400024,9.088820,0.992438,0.998148,-0.007562,düşen gün,-0.756247
2021-01-06,1657.699951,9.081770,1.002904,0.997374,0.010466,yükselen gün,1.054616
2021-01-07,1678.199951,9.006140,1.015306,0.989068,0.012402,yükselen gün,1.236653
2021-01-08,1694.400024,8.998500,1.025107,0.988229,0.009801,yükselen gün,0.965324
...,...,...,...,...,...,...,...
2022-12-23,5885.200195,19.791670,3.560530,2.173552,0.001210,yükselen gün,0.033995
2022-12-27,5876.700195,19.815269,3.555388,2.176144,-0.005142,düşen gün,-0.144430
2022-12-28,5789.399902,19.805389,3.502571,2.175059,-0.052816,düşen gün,-1.485533
2022-12-29,5915.200195,19.867519,3.578680,2.181882,0.076109,yükselen gün,2.172942


In [7]:
# index olarak kullanmak amacı ile her verinin ay ve yıl değerlerini belirttiğimiz formatta tuttuk.
Veri.index = pd.to_datetime(Veri.index) # → index alanını datetime nesnesine dönüştürdüm çünkü strftime fonksiyonuna hata aldım
Veri['ay_yıl'] = Veri.index.strftime('%Y-%m')
Veri

,Kapanış (TRY),EUR/TRY,Normalize,Normalize EUR/TRY,degisim,d_yon,degisim%,ay_yıl
2021-01-04,1652.900024,9.105680,1.000000,1.000000,NaN,yükselen gün,NaN,2021-01
2021-01-05,1640.400024,9.088820,0.992438,0.998148,-0.007562,düşen gün,-0.756247,2021-01
2021-01-06,1657.699951,9.081770,1.002904,0.997374,0.010466,yükselen gün,1.054616,2021-01
2021-01-07,1678.199951,9.006140,1.015306,0.989068,0.012402,yükselen gün,1.236653,2021-01
2021-01-08,1694.400024,8.998500,1.025107,0.988229,0.009801,yükselen gün,0.965324,2021-01
...,...,...,...,...,...,...,...,...
2022-12-23,5885.200195,19.791670,3.560530,2.173552,0.001210,yükselen gün,0.033995,2022-12
2022-12-27,5876.700195,19.815269,3.555388,2.176144,-0.005142,düşen gün,-0.144430,2022-12
2022-12-28,5789.399902,19.805389,3.502571,2.175059,-0.052816,düşen gün,-1.485533,2022-12
2022-12-29,5915.200195,19.867519,3.578680,2.181882,0.076109,yükselen gün,2.172942,2022-12


In [8]:
# Ayın son günleri için pivot table oluşturma
# Burada index olarak ayların bulunduğu sütunu aldık ve aggfunc ile de o ayın *tarihe göre sıralanmış* 
# verilerimizin en son elemanlarını yani ayın son gününün verisini alarak pivot tablosu oluşturulmuş oldu.
Veri_as = Veri.pivot_table(values = ['Kapanış (TRY)','EUR/TRY'], index='ay_yıl',aggfunc='last')
Veri_Kur = Veri_as
Veri_B30 = Veri_as
Veri_B30

,EUR/TRY,Kapanış (TRY)
ay_yıl,,
2021-01,8.902570,1572.400024
2021-02,8.904670,1556.000000
2021-03,9.748300,1430.699951
2021-04,9.923590,1456.900024
2021-05,10.347560,1506.900024
2021-06,10.384030,1462.900024
2021-07,10.050560,1500.800049
2021-08,9.869130,1588.300049
2021-09,10.343790,1505.199951


In [9]:
Veri_Kur['degisim'] = Veri_Kur['EUR/TRY'].diff(1)
Veri_Kur['d_yon'] = Veri_Kur['degisim'].map(lambda x: 'düşen gün' if x<0 else 'yükselen gün')
Veri_Kur['degisim%'] = Veri_Kur['degisim']/Veri_Kur['EUR/TRY'].shift(1)
# Veri_Kur

Veri_B30['degisim'] = Veri_B30['Kapanış (TRY)'].diff(1)
Veri_B30['d_yon'] = Veri_B30['degisim'].map(lambda x: 'düşen gün' if x<0 else 'yükselen gün')
Veri_B30['degisim%'] = Veri_B30['degisim']/Veri_B30['Kapanış (TRY)'].shift(1)
Veri_B30

,EUR/TRY,Kapanış (TRY),degisim,d_yon,degisim%
ay_yıl,,,,,
2021-01,8.902570,1572.400024,NaN,yükselen gün,NaN
2021-02,8.904670,1556.000000,-16.400024,düşen gün,-0.010430
2021-03,9.748300,1430.699951,-125.300049,düşen gün,-0.080527
2021-04,9.923590,1456.900024,26.200073,yükselen gün,0.018313
2021-05,10.347560,1506.900024,50.000000,yükselen gün,0.034319
2021-06,10.384030,1462.900024,-44.000000,düşen gün,-0.029199
2021-07,10.050560,1500.800049,37.900024,yükselen gün,0.025907
2021-08,9.869130,1588.300049,87.500000,yükselen gün,0.058302
2021-09,10.343790,1505.199951,-83.100098,düşen gün,-0.052320


2. Ay kırılımlı olarak ürettiğiniz tabloyu grafik olarak inceleyiniz.


In [10]:
px.bar(Veri_Kur, y='degisim%',
       color = 'd_yon', color_discrete_map={'düşen gün' : 'red', 'yükselen gün' : 'green'},
       text_auto = '.0%',
       title='EUR/TRY Aylık Değişim Grafiği (2021-2022)'
       )

In [11]:
px.bar(Veri_B30, y='degisim%',
       color = 'd_yon', color_discrete_map={'düşen gün' : 'red', 'yükselen gün' : 'green'},
       text_auto = '.0%',
       title='BIST30 Aylık Değişim Grafiği (2021-2022)')

# *Grafikleri inceleyecek olursak şu yorumları yapabiliriz :*

*   Öncelikle iki verinin de birbiri ile bağlantılı olduğunu daha iyi anlamak açısından iki farklı tablo yapma gereği duydum. Böylece iki farklı şekilde de aynı sonuç alacağımı tespit etmiş oldum.
*   Kur ve BIST30 endeksi 2021 yılına düşüşle başlasa da gene anlamda yükseliş trendi göstermiş olduğunu görüyoruz.
*   Aralarda düşüşler olsa da bu düşüşler maksimum %8 bandında. Bu düşüşler sonrasında hızlı bir biçimde değerini toplamış ve artış değerlerini %25'lere kadar çıkarmıştır.





3. Ders sonunda hazırladığınız 2021 ve 2022 yıllarına ait BIST30 ve EUR/USD kur değerlerini içeren bileşke tablodan sadece ayın son işgününe ait değerleri içeren bir tabloyu bir de resample yöntemi ile üretiniz.

In [19]:
# Resample yöntemi ile  de tablomuzu elde etmek mümkün. Buradaki BM(Bussines Month) ibaresi aylık bazda iş günlerini ifade etmek için kullanılan bir frekans kodudur.
# B harfi Bussines Day, M harfi ise Month anlamını taşımaktadır.
# Convention parametresi ise "pivottable()" metodunda gördüğümüz "aggfunc" parametresi ile aynı işlevi görmektedir. Bu parametre fonksiyona sıralanmış tablodaki
# ayların son iş günlerini alması talimatını verir. "asfreq()" fonksiyonu ise son değeri koruyarak herhangi bir boşluk doldurma işlemi yapmaz ve istenen haricindeki
# verileri temizler.
Veri_as2 = Veri.resample('BM', convention='end').asfreq()
Veri_as2

,Kapanış (TRY),EUR/TRY,Normalize,Normalize EUR/TRY,degisim,d_yon,degisim%,ay_yıl
2021-01-29,1572.400024,8.902570,0.951298,0.977694,-0.017847,düşen gün,-1.841563,2021-01
2021-02-26,1556.000000,8.904670,0.941376,0.977925,-0.010950,düşen gün,-1.149862,2021-02
2021-03-31,1430.699951,9.748300,0.865570,1.070573,-0.000726,düşen gün,-0.083810,2021-03
2021-04-30,1456.900024,9.923590,0.881421,1.089824,-0.005263,düşen gün,-0.593610,2021-04
2021-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-30,1462.900024,10.384030,0.885051,1.140390,-0.012160,düşen gün,-1.355359,2021-06
2021-07-30,1500.800049,10.050560,0.907980,1.103768,0.000545,yükselen gün,0.060006,2021-07
2021-08-31,1588.300049,9.869130,0.960917,1.083843,0.008651,yükselen gün,0.908516,2021-08
2021-09-30,1505.199951,10.343790,0.910642,1.135971,0.009861,yükselen gün,1.094763,2021-09
2021-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
Veri_as2['degisim'] = Veri_as2['Kapanış (TRY)'].diff(1)
Veri_as2['d_yon'] = Veri_as2['degisim'].map(lambda x: 'düşen gün' if x<0 else 'yükselen gün')
Veri_as2['degisim%'] = Veri_as2['degisim']/Veri_as2['Kapanış (TRY)'].shift(1)
Veri_as2

,Kapanış (TRY),EUR/TRY,Normalize,Normalize EUR/TRY,degisim,d_yon,degisim%,ay_yıl
2021-01-29,1572.400024,8.902570,0.951298,0.977694,NaN,yükselen gün,NaN,2021-01
2021-02-26,1556.000000,8.904670,0.941376,0.977925,-16.400024,düşen gün,-0.010430,2021-02
2021-03-31,1430.699951,9.748300,0.865570,1.070573,-125.300049,düşen gün,-0.080527,2021-03
2021-04-30,1456.900024,9.923590,0.881421,1.089824,26.200073,yükselen gün,0.018313,2021-04
2021-05-31,NaN,NaN,NaN,NaN,NaN,yükselen gün,NaN,NaN
2021-06-30,1462.900024,10.384030,0.885051,1.140390,NaN,yükselen gün,NaN,2021-06
2021-07-30,1500.800049,10.050560,0.907980,1.103768,37.900024,yükselen gün,0.025907,2021-07
2021-08-31,1588.300049,9.869130,0.960917,1.083843,87.500000,yükselen gün,0.058302,2021-08
2021-09-30,1505.199951,10.343790,0.910642,1.135971,-83.100098,düşen gün,-0.052320,2021-09
2021-10-29,NaN,NaN,NaN,NaN,NaN,yükselen gün,NaN,NaN


4. Resample yöntemi ile ürettiğiniz tabloyu grafik olarak inceleyiniz. 2. adımda elde ettiğiniz grafik ile karşılaştırarak yorumlayınız.

In [21]:
px.bar(Veri_as2, y='degisim%',
       color = 'd_yon', color_discrete_map={'düşen gün' : 'red', 'yükselen gün' : 'green'},
       text_auto = '.0%',
       title='EUR/TRY ve BIST30 Aylık Değişim Grafiği (2021-2022)'
       )

# *Grafiği inceleyecek olursak* : 

*   Genel anlamda "pivottable()" metodu ile aldığımız tablo ile aynı sonuçları aldık.
*   *Tablomuzda bazı verilerin Null olarak kaldığını gördüm. Bunun nedenini maalesef anlayamadım. Bunun nedeni metodların farklılığından olabileceği gibi benim yapmış olduğum bir hatadan da kaynaklanmış olabilir.*
*   Bu eksiklikten ötürü "pivottable()" metodu ile çıkarılan tablonun grafiği ile bu grafik arsında, bu tablodaki Null değerlerin boş bırakıldığını görebiliyoruz.



